In [1]:
from google.colab import drive   #
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pwd

/content


In [3]:
# 파일 경로를 각자 컴퓨터 환경에 맞게 수정하세요!
file_path = '/content/drive/MyDrive/부리부리데마왕/dataset/'

In [4]:
import numpy as np   # 결과해석
import pandas as pd
import sklearn
import scipy
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report,accuracy_score
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.svm import OneClassSVM
from pylab import rcParams
rcParams['figure.figsize'] = 14, 8
RANDOM_SEED = 42
LABELS = ["Normal", "Fraud"]
import plotly.graph_objs as go
import plotly
import plotly.figure_factory as ff
from plotly.offline import init_notebook_mode, iplot

In [5]:
# 데이터 불러오기
data = pd.read_csv(file_path + 'creditcard_data.csv')
data.head(15)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0
5,2.0,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,...,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080,3.67,0
6,4.0,1.229658,0.141004,0.045371,1.202613,0.191881,0.272708,-0.005159,0.081213,0.464960,...,-0.167716,-0.270710,-0.154104,-0.780055,0.750137,-0.257237,0.034507,0.005168,4.99,0
7,7.0,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,...,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339,40.80,0
8,7.0,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,...,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404,93.20,0
9,9.0,-0.338262,1.119593,1.044367,-0.222187,0.499361,-0.246761,0.651583,0.069539,-0.736727,...,-0.246914,-0.633753,-0.120794,-0.385050,-0.069733,0.094199,0.246219,0.083076,3.68,0


In [6]:
Normal = data[data['Class']==0]

In [7]:
Normal.shape

(284314, 31)

In [8]:
from imblearn.over_sampling import ADASYN

# ADASYN 모델 초기화 및 샘플링 전략 설정
adasyn = ADASYN(sampling_strategy={1: 250000}, random_state=42)

# 특징과 클래스 분리
X = data.drop('Class', axis=1)
y = data['Class']

# ADASYN을 사용하여 오버샘플링
X_resampled, y_resampled = adasyn.fit_resample(X, y)

# 새로운 데이터프레임 생성
balanced_data = pd.concat([pd.DataFrame(X_resampled, columns=X.columns), pd.Series(y_resampled, name='Class')], axis=1)

In [9]:
Fraud = balanced_data[balanced_data['Class']==1]
Fraud.shape
# Fraud.head()

(249883, 31)

In [10]:
merged_data = pd.concat([Fraud, Normal], ignore_index=True)

In [11]:
x_data = merged_data.drop('Class',axis=1)
y_data = merged_data['Class']

print(x_data.shape)
print(y_data.shape)

(534197, 30)
(534197,)


In [12]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.3,random_state = 42)



In [13]:
from sklearn.preprocessing import MinMaxScaler, QuantileTransformer
import pandas as pd
import numpy as np

def quantileTransform(data, columns):
    qt = QuantileTransformer()
    scaled_data = pd.DataFrame(qt.fit_transform(data[columns]), columns=columns)
    return scaled_data

def logTransform(data, columns):
    scaled_data = data[columns].apply(lambda x: np.log1p(x))
    return scaled_data

left_skewed_cols = ['V3', 'V5', 'V7', 'V9', 'V10', 'V12', 'V14', 'V15', 'V16', 'V17', 'V18', 'V24', 'V27', 'V28','Amount']
right_skewed_cols = ['V2', 'V11', 'V19', 'V20', 'V21']


if (x_train[left_skewed_cols].min() <= 0).any():  # .any()를 추가하여 Series 전체에서 하나라도 조건을 만족하는지 확인
    for col in left_skewed_cols:
        if x_train[col].min() <= 0:
            x_train[col] += abs(x_train[col].min()) + 0.1


# train_scaler
train_left_skewed = logTransform(x_train, left_skewed_cols)
train_right_skewed = quantileTransform(x_train, right_skewed_cols)

train_transformed = pd.concat([x_train, train_left_skewed.add_suffix('_log_Transform'), train_right_skewed.add_suffix('_qt_Transform')], axis=1)

# 그 외 수치
# minmax? standard?
from sklearn.preprocessing import MinMaxScaler
s_columns = ['V1', 'V4', 'V6', 'V8', 'V13', 'V22', 'V23', 'V25', 'V26']
minmaxScaler = MinMaxScaler()

for col in s_columns:
    train_transformed[col + '_MM_Transform'] = minmaxScaler.fit_transform(train_transformed[[col]])

#train_transformed --> 전처리가 진행된 데이터
train_transformed.drop(columns ='Time', inplace = True)

In [14]:
from sklearn.preprocessing import MinMaxScaler
#꼬리가 오른쪽으로 긴 변수
pro_skew = ['V3', 'V5', 'V7','V9', 'V10', 'V12', 'V14', 'V15', 'V16', 'V17', 'V18', 'V24','V27', 'V28']
neg_skew = ['V2', 'V11', 'V19', 'V20', 'V21']
normal_col = ['V1', 'V4','V6', 'V8', 'V13', 'V22', 'V23', 'V25', 'V26' ]

# pro_skew 로그변환 적용
# 변수 내의 최소값이 0 이하인 경우, 모든 값을 조금 올려서 로그 변환이 가능하도록 조정
if (data[pro_skew].min() <= 0).any():  # .any()를 추가하여 Series 전체에서 하나라도 조건을 만족하는지 확인
    for col in pro_skew:
        if data[col].min() <= 0:
            data[col] += abs(data[col].min()) + 0.1

for col in pro_skew:
    data[col + '_log_transformed'] = np.log(data[col])


# neg_skew는 음의 왜도.
# 제곱 변환 적용
for col in neg_skew:
  data[col + '_squared_transformed'] = data[col] ** 2

#normal_col
# 정규화할 변수 리스트
scaler = MinMaxScaler()
for col in normal_col:
    data[col] = scaler.fit_transform(data[col].values.reshape(-1, 1)) # 2차원 배열로 바꿔서 실행


# 적용 다한 column은 drop
data.drop(columns = pro_skew+neg_skew+normal_col, inplace = True)

data.head()

,Time,Amount,Class,V3_log_transformed,V5_log_transformed,V7_log_transformed,V9_log_transformed,V10_log_transformed,V12_log_transformed,V14_log_transformed,...,V17_log_transformed,V18_log_transformed,V24_log_transformed,V27_log_transformed,V28_log_transformed,V2_squared_transformed,V11_squared_transformed,V19_squared_transformed,V20_squared_transformed,V21_squared_transformed
0,0.0,149.62,0,3.931079,4.731847,3.781842,2.631734,3.209999,2.899547,2.944605,...,3.237532,2.264316,1.099797,3.126727,2.741422,0.005297,0.304262,0.163210,0.063208,0.000335
1,0.0,2.69,0,3.883460,4.735350,3.774563,2.586157,3.199542,2.988151,2.953375,...,3.224778,2.242345,0.954272,3.120455,2.743727,0.070836,2.600887,0.021253,0.004772,0.050974
2,1.0,378.66,0,3.915991,4.730393,3.794336,2.486523,3.214703,2.936502,2.952218,...,3.272332,2.248909,0.809750,3.118407,2.738924,1.796037,0.390002,5.115998,0.275604,0.061503
3,1.0,123.50,0,3.916385,4.734732,3.781797,2.497086,3.204100,2.942434,2.945828,...,3.201880,2.447942,0.565911,3.123615,2.746729,0.034309,0.051296,1.519357,0.043280,0.011729
4,2.0,69.99,0,3.911509,4.731240,3.789859,2.663876,3.236375,2.961240,2.901128,...,3.219906,2.257645,1.124246,3.130486,2.756538,0.770422,0.677070,0.645591,0.166907,0.000089


In [15]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
SEED = 42

# Split the data into features and target variable
X = data.drop(columns=[ 'Class'])  # Features
y = data['Class']                       # Target variable

# Split the data into a train+validation set and test set
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=SEED)

# Further split the train+validation set into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=SEED) # 0.25 x 0.8 = 0.2

In [16]:
from sklearn.naive_bayes import GaussianNB


model = GaussianNB()   # 가우시안 naive Bayes
model.fit(X_train , y_train)

GaussianNB()

In [17]:
from sklearn import metrics   #F1score가 굉장히 낮게 나옴,
from sklearn.metrics import accuracy_score, f1_score

expected = y_val
predicted = model.predict(X_val)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56855
           1       0.19      0.61      0.29       106

    accuracy                           0.99     56961
   macro avg       0.59      0.80      0.64     56961
weighted avg       1.00      0.99      1.00     56961

Accuracy: 0.9942943417425958
F1 Score: 0.2857142857142857


In [18]:
from sklearn.metrics import confusion_matrix
# 그 다른것도 다

# 혼동 행렬 얻기
conf_matrix = confusion_matrix(expected, predicted)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity   굉장히 낮음
Sensitivity = tp / (tp + tn)

#Specision  굉장히 낮음
Specision = fn / (fp + fn)

# Precision  굉장히 낮음
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)



print('False Positive Rate:', fpr)
print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Negative Rate : ',tnr )

False Positive Rate: 0.0049951631342889805
Sensitivity 0.0011476799208983686
Specision 0.12615384615384614
Precision 0.18624641833810887
False Negative Rate :  0.9988523200791016


In [19]:

from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB(alpha=2.0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("정확도:", accuracy_score(y_test, y_pred))  # 가우시안 보다 높게 나옴.

정확도: 0.9983848881710614


In [20]:
from sklearn import metrics   #F1score가 굉장히 낮게 나옴,
from sklearn.metrics import accuracy_score, f1_score

expected = y_test
predicted = model.predict(X_test)   #accuracy는 굉장히 높게 나오는데 F1score값이 너무 낮음.
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)


              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56867
           1       0.80      0.04      0.08        95

    accuracy                           1.00     56962
   macro avg       0.90      0.52      0.54     56962
weighted avg       1.00      1.00      1.00     56962

Accuracy: 0.9983848881710614
F1 Score: 0.07999999999999999


In [21]:
from sklearn.metrics import confusion_matrix    # 베르누이 나이브 베이즈 분류
# 그 다른것도 다
from sklearn.naive_bayes import BernoulliNB

model = BernoulliNB(alpha=2.0)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


# 혼동 행렬 얻기
conf_matrix = confusion_matrix(y_test, y_pred)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)



print('False Positive Rate:', fpr)
print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Negative Rate : ',tnr )

False Positive Rate: 1.7584891061599872e-05
Sensitivity 7.03358537014243e-05
Specision 0.9891304347826086
Precision 0.8
False Negative Rate :  0.9999296641462986


In [22]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(X_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression



LogisticRegression()

In [23]:
from sklearn.metrics import accuracy_score  # 로지스틱도 결과가 좋게 나옴,

y_pred = model.predict(X_train)
accuracy_score(y_train, y_pred)

0.9986072341894747

In [24]:
from sklearn import metrics
from sklearn.metrics import accuracy_score, f1_score  # 여기서도 로지스틱 결과는 좋게 나오지 않음.

expected = y_val
predicted = model.predict(X_val)
f1 = f1_score(expected, predicted)

print(metrics.classification_report(expected, predicted))
print('Accuracy:', accuracy_score(expected, predicted))
print('F1 Score:', f1)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00     56855
           1       0.86      0.23      0.36       106

    accuracy                           1.00     56961
   macro avg       0.93      0.61      0.68     56961
weighted avg       1.00      1.00      1.00     56961

Accuracy: 0.998490195045733
F1 Score: 0.3582089552238806


In [25]:
from sklearn.metrics import confusion_matrix    # 로지스틱 회귀
# 로지스틱일 때의 이 값들이 가장 좋게 나옴.

model = LogisticRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)


# 혼동 행렬 얻기
conf_matrix = confusion_matrix(y_test, y_pred)

# False Positive, True Negative 값을 추출
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# False Positive, False Negative
tn, fp, fn, tp = conf_matrix.ravel()
# True Positive , True Negative
tn, fp, fn, tp = conf_matrix.ravel()


# FPR 계산
fpr = fp / (fp + tn)

#Sensitivity
Sensitivity = tp / (tp + tn)

#Specision
Specision = fn / (fp + fn)

# Precision
pre = tp / (tp + fp)

#FNR
tnr = tn / (tp + tn)



print('False Positive Rate:', fpr)
print('Sensitivity',Sensitivity )
print('Specision',Specision )
print('Precision', pre)
print('False Negative Rate : ',tnr )

False Positive Rate: 8.792445530799937e-05
Sensitivity 0.00036917884077843993
Specision 0.9367088607594937
Precision 0.8076923076923077
False Negative Rate :  0.9996308211592215


/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning:

lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression

